In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Training the MNIST dataset with Artificial Neural Networks
With the aim of comparing the performance of training convolutional neural networks and artificial neural networks a training of the MNIST dataset is performed with both techniques. 

## Transform MNIST images to tensors

In [ ]:
# instantiate the transformer from image to tensor
transform = transforms.ToTensor()

train_data = datasets.MNIST(root="train_images/MNIST", train=True, transform=transform, download=True)